In [1]:
import pandas as pd 

In [2]:
import numpy as np

In [3]:
data = pd.read_csv("./PlayTennis_.csv", )

In [4]:
data.drop('Day', axis=1, inplace=True)

In [5]:
data.head()

,Outlook,Temperature,Humidity,Wind,Playtennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [6]:
rows = data.shape[0]

In [7]:
def getEntropyGlobal(d, y):
    daf = d.groupby([y]).size().reset_index(name='S')
    S_Entropy_df = daf.pivot_table( columns = y, values = "S",fill_value = 0)
    S_Entropy_df['total'] = S_Entropy_df['No'] + S_Entropy_df['Yes']
    epsilon = 1e-9
    S_Entropy_df['entropy'] = (-(
        (S_Entropy_df['Yes']+epsilon)/(S_Entropy_df['total']+2*epsilon)*np.log2((S_Entropy_df['Yes']+epsilon)/(S_Entropy_df['total']+2*epsilon)) 
        + 
        (S_Entropy_df['No']+epsilon)/(S_Entropy_df['total']+2*epsilon)*np.log2((S_Entropy_df['No']+epsilon)/(S_Entropy_df['total']+2*epsilon))
    )).round(4)
    return S_Entropy_df

In [8]:
def get_S_Entropy_Gain(dataframe, x, y, es):
    d = dataframe[[x,y]]
    rows = d.shape[0]
    daf = d.groupby([x,y]).size().reset_index(name='S')
    S_Entropy_df = daf.pivot_table(index = x, columns = y, values = "S",fill_value = 0)
    S_Entropy_df['total'] = S_Entropy_df['No'] + S_Entropy_df['Yes']    
    epsilon = 1e-9
    S_Entropy_df['entropy'] = (-(
        (S_Entropy_df['Yes']+epsilon)/(S_Entropy_df['total']+2*epsilon)*np.log2((S_Entropy_df['Yes']+epsilon)/(S_Entropy_df['total']+2*epsilon)) 
        + 
        (S_Entropy_df['No']+epsilon)/(S_Entropy_df['total']+2*epsilon)*np.log2((S_Entropy_df['No']+epsilon)/(S_Entropy_df['total']+2*epsilon))
    ))
    S_Entropy_df['gain_1'] = (
        (S_Entropy_df['total'] / rows) * S_Entropy_df['entropy']
    )
    gain = (es['entropy'] - S_Entropy_df['gain_1'].sum()).rename(x)
    print(S_Entropy_df)
    return gain, S_Entropy_df

In [9]:
def find_Node(data):
    y = data.columns[-1]
    x = data.columns[:-1]
    es = getEntropyGlobal(data, y)
    print('S : ', es)
    dicti = {'feat': [], 'gain':[]}
    for elem in x:
        j, k = get_S_Entropy_Gain(data, elem, y, es)
        dicti['feat'].append(j.name)
        dicti['gain'].append(j[0])
    # print(dicti)
    df = pd.DataFrame(dicti) 
    print('Gain df: \n',df)
    return df.iloc[df['gain'].idxmax()]['feat']

first_Node = find_Node(data)
first_Node

S :  Playtennis  No  Yes  total  entropy
S            5    9     14   0.9403
Playtennis  No  Yes  total       entropy        gain_1
Outlook                                               
Overcast     0    4      4  8.335012e-09  2.381432e-09
Rain         2    3      5  9.709506e-01  3.467681e-01
Sunny        3    2      5  9.709506e-01  3.467681e-01
Playtennis   No  Yes  total   entropy    gain_1
Temperature                                    
Cool          1    3      4  0.811278  0.231794
Hot           2    2      4  1.000000  0.285714
Mild          2    4      6  0.918296  0.393555
Playtennis  No  Yes  total   entropy    gain_1
Humidity                                      
High         4    3      7  0.985228  0.492614
Normal       1    6      7  0.591673  0.295836
Playtennis  No  Yes  total   entropy    gain_1
Wind                                          
Strong       3    3      6  1.000000  0.428571
Weak         2    6      8  0.811278  0.463587
Gain df: 
           feat      g

'Outlook'